#  Bio 208: Working with tabular data in Pandas

## Pandas library

[Pandas](https://pandas.pydata.org/) is a widely used Python library for working with tabular data.

The usual convention that Pandas adopts when working with tabular data is one in which the rows of the data represent the "cases". "observations", or "entities" we're studying (e.g. individuals in a population, genomic features, geographical regions, etc) and the columns of the table represent the variables of interest that have been determined or measured or recorded for those cases or entities (e.g. gene identifiers, sequence length, measures of expression, etc). 

![Image from Pandas tutorial.](https://pandas.pydata.org/docs/_images/01_table_dataframe.svg)

In [ ]:
import pandas as pd
import numpy as np  # we'll import numpy as well, as it will be useful for calculations

### Creating DataFrames

The core data structure in the Pandas library is called a "data frame" (`DataFrame`).

We can create a dataframe by calling the `DataFrame` class with a dictionary in which the keys are the names of the columns, and the values are lists or arrays of the values in each column. This is illustrated below:

In [ ]:
df = pd.DataFrame({
    "Name": ["ORF1ab", "S", "E", "M", "N", "YFG1"],
    "Start": [266,  21563, 26245, 26523, 28274,99],
    "Stop": [21555, 25384, 26472, 27191, 29533, 150],
    "Product": ["ORF1ab polyprotein", "surface glycoprotein", 
                "envelope protein", "membrane glycoprotein",
                "nucleocapsid phosphoprotein", pd.NA]
})

In [ ]:
df

### Reading a DataFrame from a file

Usually we won't be creating DataFrames by hand but rather reading tabular data from files. Pandas include a variety of `read_*` functions such as `read_csv` and `read_excel` for reading such data. We'll primarily using `read_csv` in this class, to read tables delimited with either commas ("comma separated values" = `.csv` files) or tabs ("tab separated values" = `.tsv` files).

In [ ]:
# read a test spreadsheet I created in Excel durig our class session
df2 = pd.read_table("/Users/pmagwene/Downloads/example-table.tsv")  

In [ ]:
df2

### What are the names and types of the columns of the DataFrame. What are the dimensions (number of rows and columns) of the DataFrame?

In [ ]:
# return the column names
df.columns

In [ ]:
df.dtypes

In [ ]:
df.shape

The `len` function applied to a data frame gives the number of rows.

In [ ]:
len(df)

### Getting a specific column from a DataFrame

To retrieve a specific column of data from a DataFrame we can index the DataFrame with a string that gives the column name of interest.

In [ ]:
df["Product"]

When a column name would also be a valid Python variable name, there is a short-hand way to access columns, as if they were attributes of the DataFrame, like so:

In [ ]:
df.Product

### Getting specific rows from a DataFrame using slices

When you index a DataFrame with an integer slice instead of a string, this returns the specified range of rows (obeying Python's standard 0-indexing conventions).  Not that row indexing like this requires a slice; a single integer index won't work with DataFrames.

In [ ]:
df[0:2]  # get the first two rows

In [ ]:
df[:1]  # to get the first row I still had to use the slice syntax; df[0] won't work

### Getting  subset of rows and columns from a DataFrame

If you want to simultaneously get a subset of rows and/or columns, you can use the `DataFrame.loc` (location) attribute. 

One subtle but important point about using `loc` is that when specifing the row slice, the labels correspond not to the integer position along the rows but rather the indexing labels.  When you create a DataFrame, you can specify a set of index labels for the rows.  If you don't specify such labels, Pandas will create a set of default indexing labels based on the integer positions.  This is what is being shown by the numbers in bold on the left margin of the data frames when we display them in this notebook.  The `loc` attribute allows us to create slices based on these row index labels, but these these index label slices differ from the positional slices in that they are inclusive (both start and stop labels are included).  I'll do my best to illustrate how this works below with some examples.

First let's see how to get a specific subset of rows and columns using `loc`:

In [ ]:
# get rows with the index labels 3 and 1, and columns "Product" and "Start"
df.loc[[3, 1], ["Product","Start"]]  

We can slice ranges of rows and columns using `loc`:

In [ ]:
# all rows, columns from Name to Stop
df.loc[:, "Name":"Stop"]

However if we specify a slice on the rows, notice how it uses the label indices on the left to specify what rows to take, and also notice that the slice is inclusive of the last specified row (if this was instead based on Python's standard positional indexing the statement below would be expected to return only two rows):

In [ ]:
df.loc[1:3, ["Name","Product"]]

The subtleties of row-indexing with `loc` become particularly apparent if we create a version of the table in which we've reversed the row order.

In [ ]:
df_reversed = df[::-1]  # get the rows in reversed order
df_reversed

With this reversed version of the data frame, the same slice syntax we used above doesn't work because the row label indices are no longer in the order 1 to 3.

In [ ]:
# returns an empty data frame because the label indices 1:3 won't work
df_reversed.loc[1:3, ["Name", "Product"]] 

However specifying the slice range as `3:1` works because that is consistent with the order of labelled indices in `df_reversed`:

In [ ]:
# returns an empty data frame because the label indices 1:3 won't work
df_reversed.loc[3:1, ["Name", "Product"]] 

### Selecting cross sections of a DataFrame by integer positions using `DataFrame.iloc`

`DataFrame.iloc` eanbles conventional indexing of both rows and columns by integer position, as illustrated below:

In [ ]:
df.iloc[:3, :2]

In [ ]:
df.iloc[0,2]  # value in the first row, third column

In [ ]:
df.iloc[[2,0],[0,3]]

In [ ]:
df_reversed.iloc[1:3, [0,3]]

### Creating new columns by computing on existing columns

DataFrames support a simple syntax for creating new columns from existing ones.  Like numpy arrays, most operations with data frame columns work element-by-element so we can calculate new variables of interest from existing ones by applying functions or operations to one or more of the variables.

Here I show how to create a new column "Length" by subtracting the Start coordinates from each of the Stop coordinates:

In [ ]:
df["Length"] = df.Stop - df.Start + 1
df

### Subsetting the rows of a DataFrame by Boolean indexing

When working with large data sets, we frequently want to explore how variables of interest differ across different subsets of the cases.  Pandas (and Numpy) facilitate this sort of analysis by allowing us to subset rows of a DataFrame using Boolean indexing.

To illustrate this we'll look at ways to subset our small data frame to get at genes that meet certain criteria.

The COVID-19 genome is about 30 Kbp in length.  Consider the case where we want to find only those genes towards the "right" end of the genome (i.e. with respect to the arbitrary coordinate system the reference genome has been assigned is reported).  One way to do this would be to find all those genes for which the "Start" value is greater than some cutoff, say 25,000.

Let's see what happens when we compare each of the values (rows) in the "Start" column and asks whether the corresponding value is greater than 25000.  

In [ ]:
df.Start > 25000

As you see, we get back a column (Series) of Boolean (True/False) values indicating for which of the corresponding elements the comparison is True.

Since that statement return a Boolean series, we can use it directly to index the rows of our data frame. You might find it useful to read think about the following Python case as saying "df where df.Start is greater than 25000".

In [ ]:
df[df.Start > 25000]

Boolean indexing like this creates a new DataFrame, which we'd typically we'd assign to a variable so would do further computations with it.

In [ ]:
endgenes = df[df.Start > 25000]

In [ ]:
endgenes.loc[:,["Name","Product"]]

### More complex subsetting using Boolean operators

Pandas defines the logical operators `&` (and) and `|` (or) for working with Boolean Series.  For example, the following returns a Boolean Series corresponding to the rows of our data frame where the start position was greater than 25,000 and the gene length was greater than 500.

In [ ]:
end_and_long = (df.Start > 25000) & (df.Length > 500)
end_and_long

Again, the final result is a single Boolean series which we can use to filter or subset the rows of our data frame:

In [ ]:
df[end_and_long]

### Creating plots from DataFrames

When generating plots from data in Pandas DataFrames you can pass variables of interest to the respective `matplotlib` functions or use plotting attributes associated with the DataFrame object (which in turn uses matplotlib as it's backend).  I illustrate both approaches below.

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# generate a histogram of genes Lengths
plt.hist(df.Length)

pass # adding pass here prevents the data objects returned from 
     # plt.hist from being printed in the output. This is just to make
     # the notebook output look neat and tidy.

Using the `plot` attribute associated with DataFrames:

In [ ]:
df.Length.plot.hist()

### Summary statistics from DataFrames

Numerical columns in DataFrame support built-in methods for calculating summary statistics:

In [ ]:
df.Length.mean()  # average length of the genes

In [ ]:
df.Length.median() # median length of the genes

In [ ]:
df.Length.describe()  # mulitiple summary statistics about the values in the Length column

## Working with a table of features from the Saccharomyces Genome Database (SGD)

The file [`SGD_features.tsv`](https://github.com/bio208fs-class/bio208fs-lecture/raw/master/data/SGD_features.tsv) is a tab-delimited file I downloaded from SGD that summarizes key pieces of information about genome features in the budding yeast genome.  The original file can be found here: http://sgd-archive.yeastgenome.org/curation/chromosomal_feature/

Here's a short summary of the contents of this file, from the "SGD_features.README" document:

```
1. Information on current chromosomal features in SGD, including Dubious ORFs. 
Also contains coordinates of intron, exons, and other subfeatures that are located within a chromosomal feature.

2. The relationship between subfeatures and the feature in which they
are located is identified by the feature name in column #7 (parent
feature). For example, the parent feature of the intron found in
ACT1/YFL039C will be YFL039C. The parent feature of YFL039C is
chromosome 6.

3. The coordinates of all features are in chromosomal coordinates.

Columns within SGD_features.tab:

1.   Primary SGDID (mandatory)
2.   Feature type (mandatory)
3.   Feature qualifier (optional)
4.   Feature name (optional)
5.   Standard gene name (optional)
6.   Alias (optional, multiples separated by |)
7.   Parent feature name (optional)
8.   Secondary SGDID (optional, multiples separated by |)
9.   Chromosome (optional)
10.  Start_coordinate (optional)
11.  Stop_coordinate (optional)
12.  Strand (optional)
13.  Genetic position (optional)
14.  Coordinate version (optional)
15.  Sequence version (optional)
16.  Description (optional)

Note that "chromosome 17" is the mitochondrial chromosome.
```


Download [`SGD_features.tsv`](https://github.com/bio208fs-class/bio208fs-lecture/raw/master/data/SGD_features.tsv) to your computer and then load it using the `read_csv` function, specifying the delimiter argument as a tab:

In [ ]:
features = pd.read_csv("/Users/pmagwene/Downloads/SGD_features.tsv", delimiter="\t")

## What are the dimensions of this data set?

In [ ]:
features.shape

## What are the columns names and data types?

In [ ]:
features.columns

In [ ]:
features.dtypes

## How many genome features are there in the  yeast genome?

In [ ]:
len(features)  

## What are the different feature types?

In [ ]:
features.Type.unique()  # gives unique elements in a column

In [ ]:
len(features.Type.unique())

## Check the chromosome designations

In [ ]:
features.Chromosome.unique()

## Adding a new column - Length

In [ ]:
# using np.abs here because for some of the features Start > Stop
# The +1 accounts for the fact that the start/stop coordinates are inclusive
features["Length"] = np.abs(features.Stop - features.Start) + 1

In [ ]:
features.Length.dtype

In [ ]:
# create a histogram showing distribution of lengths of features
features.Length.plot.hist(bins=100)
pass

## How  many of those features are annotated as "ORFs" (open reading frames)?

In [ ]:
orfs = features[features.Type == "ORF"]
len(orfs)

## Sorting ORFs based on their length

In [ ]:
orfs.sort_values("Length").loc[:,["SGDID", "Chromosome",
                                  "Gene","Length","Description"]].head()

In [ ]:
# sort in descending order
orfs.sort_values("Length", ascending=False).loc[:,["SGDID", "Chromosome",
                                                   "Gene","Length","Description"]].head()

## Convert the Chromosome data type from object to numerical type

In [ ]:
# drop the 2-micron from consideration
orfs = orfs[orfs.Chromosome != "2-micron"].copy() 

# we use copy above because we're going to make some modifications of the ORF data
# so we want an independent copy of the data not a "view" into the features DataFrame

In [ ]:
orfs.loc[:,"Chromosome"] = pd.to_numeric(orfs.Chromosome)

In [ ]:
orfs.Chromosome.unique()

## Distribution of ORF lengths

In [ ]:
plt.hist(orfs.Length, bins=100)
plt.xlabel("Gene length")
plt.ylabel("Frequency")
pass

Since lengths differ by several orders of magnitude, a log-transform might be useful

In [ ]:
plt.hist(np.log10(orfs.Length),bins = 100)
plt.xlabel("log10(Gene length)")
plt.ylabel("Frequency")
pass

### How many of the ORFS are designated as "Dubious"? How many are "Verified"?

In [ ]:
dubious = orfs[orfs.Qualifier == "Dubious"]
len(dubious)

In [ ]:
verified = orfs[orfs.Qualifier == "Verified"]
len(verified)

### What is the distribution of length of dubious ORFs? What is the distribution of lengths of verified ORFs?

In [ ]:
plt.hist(verified.Length, alpha=0.5, bins=np.arange(0,12000,100))
plt.hist(dubious.Length, alpha=0.5, bins=np.arange(0,12000,100))

# add labels
plt.xlabel("ORF length (bp)")
plt.ylabel("Frequency")
pass


In [ ]:
dubious.Length.median(), verified.Length.median()

In [ ]:
# here we're looking at the the log10 of the frequencies NOT the lengths

# generate histograms
plt.hist(verified.Length, color='red', alpha=0.35, bins=np.arange(0,12000,100),log=True)
plt.hist(dubious.Length, color='black', alpha=0.35, bins=np.arange(0,12000,100), log=True)

# add dashed lines representing median lengths
plt.vlines(dubious.Length.median(), ymin=0, ymax=400, color='black', linestyle="dashed")
plt.vlines(verified.Length.median(), ymin=0, ymax=400, color='red', linestyle="dashed")

# add labels
plt.xlabel("ORF length (bp)")
plt.ylabel("Log10(Frequency)")
pass

## Dubious + Verified is not the full set of ORFS. What are the other Qualifier values?

In [ ]:
orfs.Qualifier.unique()

## Using groupby to get a breakdown of ORFs

In [ ]:
orfs.groupby("Qualifier").Qualifier.count()

In [ ]:
orfs.Qualifier.value_counts()  # short hand for counting things in categories

## Using the groupby method to calculate aggregate statistics

In [ ]:
orfs.groupby("Qualifier").Length.median()

## Grouping by multiple variables

In [ ]:
orfs.groupby(["Chromosome", "Qualifier"]).Qualifier.count().head(6)
# table continues on but I just took the first six values to truncate output

## Visualizations of Chromosomes

In [ ]:
# load chromosome length data
chroms = pd.read_csv("/Users/pmagwene/Downloads/chromosome_length.tsv", 
                     delimiter="\t")

In [ ]:
chroms.head()

## Plot the ORFS on a single chromosome

In [ ]:
# plot chromosome 1
plt.barh(1, chroms.Length[chroms.Chromosome == 1], height=0.25, color='steelblue')

chr01orfs = orfs[orfs.Chromosome == 1]

for (start, stop) in zip(chr01orfs.Start, chr01orfs.Stop):
    left = min(start, stop)
    width = abs(stop - start)
    plt.barh(1, width, left=left, color='orange', height=0.35)


plt.xlabel("Position (bp)")
plt.ylabel("Chromosome")
    
# specify specific y-axis scaling to make this look nice
plt.ylim(0,2)
pass

In [ ]:
# plot chromosome 1 and 2 orfs

plt.barh(1, chroms.Length[chroms.Chromosome == 1], height=0.25, color='steelblue')
plt.barh(2, chroms.Length[chroms.Chromosome == 2], height=0.25, color='steelblue')

focalorfs = orfs[orfs.Chromosome.isin([1,2])]

for (chrom, start, stop) in zip(focalorfs.Chromosome, focalorfs.Start, focalorfs.Stop):
    left = min(start, stop)
    width = abs(stop - start)
    plt.barh(chrom, width, left=left, color='orange', height=0.35)
    
plt.xlabel("Position (bp)")
plt.ylabel("Chromosome")
    
# specify specific y-axis scaling to make this look nice
plt.ylim(0,3)
pass